In [1]:
import os
import sys
import json
import pickle
import random
from itertools import chain

import numpy as np
import pandas as pd
from tqdm.notebook import trange

import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import layers
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


class MPCDataset:
    def __init__(self, data_dir, info_dir='./info/', **kwargs):
        
        self.data_dir = data_dir if data_dir[-1] == '/' else data_dir + '/'
        self.info_dir = info_dir if info_dir[-1] == '/' else info_dir + '/'
        
        self.train_plylst_list = None
        self.val_plylst_list = None
        self.test_plylst_list = None
        
        self.song_tag_d2v = None
        self.tag_song_d2v = None
        
        self.tag_maxlen = None
        self.n_tags = None
        self.idx2tag = None
        self.tag2idx = None
        
        self.song_maxlen = None
        self.n_songs = None
        self.idx2song = None
        self.song2idx = None
        
        self.song_vectors = None
        self.tag_vectors = None
        
        self.song_window=100
        self.tag_window=5
        self.min_count=2
        self.negative=5
        self.worker=4
        self.vector_size=256
        for k,v in kwargs.items():
            if k == 'song_window':
                song_window=v
            if k == 'tag_window':
                tag_window=v
            if k == 'min_count':
                min_count=v
            if k == 'negative':
                negative=v
            if k == 'worker':
                worker=v
            if k == 'vector_size':
                vector_size=v
        
        if not os.path.isdir(self.info_dir):
            os.mkdir(self.info_dir)

        self.train_plylst_list = self.load_json(
            os.path.join(self.data_dir, 'train.json'))
        self.val_plylst_list = self.load_json(
            os.path.join(self.data_dir, 'val.json'))
        self.test_plylst_list = self.load_json(
            os.path.join(self.data_dir, 'test.json'))
        
        # load song_tag_d2v, tag_song_d2v
        self.song_tag_d2v, self.tag_song_d2v = self.get_d2v_models()    
        
        self.idx2song = ['<pad>', '<unk>'] + self.song_tag_d2v.wv.index2word
        self.song2idx = {song:idx for idx, song in enumerate(self.idx2song)}
        
        self.idx2tag = ['<pad>', '<unk>'] + self.tag_song_d2v.wv.index2word
        self.tag2idx = {tag:idx for idx, tag in enumerate(self.idx2tag)}
        
        self.song_maxlen, self.tag_maxlen = self.get_maxlen()
        self.n_songs = len(self.idx2song)
        self.n_tags = len(self.idx2tag)
        
        song_vectors_path = self.info_dir + 'song_vectors.npy'
        tag_vectors_path = self.info_dir + 'tag_vectors.npy'
        if not (os.path.isfile(song_vectors_path) and os.path.isfile(tag_vectors_path)):
            self.song_vectors = np.concatenate([np.zeros((2, self.vector_size)), 
                                  self.song_tag_d2v.wv.vectors], axis=0)
            self.tag_vectors = np.concatenate([np.zeros((2, self.vector_size)), 
                                  self.tag_song_d2v.wv.vectors], axis=0)
            
            np.save(song_vectors_path, self.song_vectors)
            np.save(tag_vectors_path, self.tag_vectors)
            
        elif os.path.isfile(song_vectors_path) and os.path.isfile(tag_vectors_path):
            self.song_vectors = np.load(song_vectors_path)
            self.tag_vectors = np.load(tag_vectors_path)

    def load_json(self, path):
        return json.load(open(path, 'r'))
    
    def get_d2v_models(self):
        song_tag_d2v_path = self.info_dir + 'song_tag_d2v.model'
        tag_song_d2v_path = self.info_dir + 'tag_song_d2v.model'
        
        song_tag_d2v = None
        tag_song_d2v = None
        if not(os.path.isfile(song_tag_d2v_path) and os.path.isfile(tag_song_d2v_path)):
            song_tag_doc_list = list()
            tag_song_doc_list = list()
            for plylst in chain(self.train_plylst_list, 
                                self.val_plylst_list, 
                                self.test_plylst_list):
                songs = list()
                for song in plylst['songs']:
                    songs.append(str(song))
                    
                tags = list()
                for tag in plylst['tags']:
                    tags.append(str(tag))
                    
                song_tag_doc_list.append(TaggedDocument(songs, tags))
                tag_song_doc_list.append(TaggedDocument(tags, songs))

            song_tag_d2v = Doc2Vec(song_tag_doc_list, window=self.song_window, 
                                   min_count=self.min_count, negative=self.negative, 
                                   worker=self.worker, vector_size=self.vector_size)
            tag_song_d2v = Doc2Vec(tag_song_doc_list, window=self.tag_window, 
                                   min_count=self.min_count, negative=self.negative, 
                                   worker=self.worker, vector_size=self.vector_size)
            
            song_tag_d2v.save(song_tag_d2v_path)
            tag_song_d2v.save(tag_song_d2v_path)

            
        elif os.path.isfile(song_tag_d2v_path) and os.path.isfile(tag_song_d2v_path):
            song_tag_d2v = Doc2Vec.load(song_tag_d2v_path)
            tag_song_d2v = Doc2Vec.load(tag_song_d2v_path)
        
        return song_tag_d2v, tag_song_d2v
    
    def get_maxlen(self):
        song_maxlen = -1
        tag_maxlen = -1
        for plylst in chain(self.train_plylst_list, 
                            self.val_plylst_list, 
                            self.test_plylst_list):
            song_maxlen = max(song_maxlen, len(plylst['songs']))
            tag_maxlen = max(tag_maxlen, len(plylst['tags']))
            
        return song_maxlen, tag_maxlen
    
    def generate_input(self, mode, batch_size, callback):
        def _encode(plylst, feature, mode):
            
            if feature == 'tag':
                feat_idxs = list()
                for tag in plylst['tags']:
                    if self.tag2idx.setdefault(str(tag), False):
                        feat_idxs.append(self.tag2idx[str(tag)])
                    else:
                        feat_idxs.append(self.tag2idx['<unk>'])
                feat_maxlen = self.tag_maxlen
                n_feats = self.n_tags
            else:
                feat_idxs = list()
                for song in plylst['songs']:
                    if self.song2idx.setdefault(str(song), False):
                        feat_idxs.append(self.song2idx[str(song)])
                    else:
                        feat_idxs.append(self.song2idx['<unk>'])
                feat_maxlen = self.song_maxlen
                n_feats = self.n_songs

            # feat_included
            if mode == 'train':
                random.shuffle(feat_idxs)
                n_included = len(feat_idxs) // 2
            else:
                n_included = len(feat_idxs)
            feat_included = feat_idxs[:n_included]

            # feat_input: padded label encoding(feat_included)
            feat_input = np.zeros((feat_maxlen,), dtype=np.int32)
            feat_input[:n_included] = np.array(feat_included)

            # feat_label: one-hot encoding(feat_included & feat_excluded)
            feat_label = np.zeros((n_feats,), dtype=np.float64)
            feat_label[feat_idxs] = 1
            
            return feat_input, feat_label
        
        if mode == 'train':
            plylst_list = self.train_plylst_list
        elif mode == 'val':
            plylst_list = self.val_plylst_list
        elif mode == 'test':
            plylst_list = self.test_plylst_list
        else:
            raise(ValueError)
            
        id_list = list()
        song_input_list = list()
        tag_input_list = list()
        song_label_list = list()
        tag_label_list = list()
        for i, plylst in enumerate(plylst_list, 1):
            id_ = plylst['id']
            id_list.append(id_)
            song_input, song_label = _encode(plylst, 'song', mode)
            tag_input, tag_label = _encode(plylst, 'tag', mode)
            song_input_list.append(song_input)
            tag_input_list.append(tag_input)
            song_label_list.append(song_label)
            tag_label_list.append(tag_label)
            
            if (i%batch_size == 0) or (i == len(plylst_list)):
                ids = np.array(id_list, dtype=np.int32)
                song_inputs = np.stack(song_input_list, axis=0)
                tag_inputs = np.stack(tag_input_list, axis=0)
                id_list = list()
                song_input_list = list()
                tag_input_list = list()
                
                song_labels = np.stack(song_label_list, axis=0)
                tag_labels = np.stack(tag_label_list, axis=0)
                song_label_list = list()
                tag_label_list = list()
                
                yield callback(ids, (song_inputs, tag_inputs), (song_labels, tag_labels))

In [2]:
# from .dataset import MPCDataset
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append("/content/drive/My Drive/MelonPlaylistContinuation/")


DATA_DIR = "/content/drive/My Drive/MelonPlaylistContinuation/data/"
INFO_DIR = "/content/drive/My Drive/MelonPlaylistContinuation/info/"
CKPT_DIR = "/content/drive/My Drive/MelonPlaylistContinuation/checkpoint/d2v_dense/"
if not os.path.isdir(CKPT_DIR):
    os.mkdir(CKPT_DIR)
SEED = 200722
BS = 256
EPOCHS = 10
dataset = MPCDataset(DATA_DIR, INFO_DIR, vector_size=256)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
def preprocess_songs(ids, inputs, labels):
    (song_inputs, tag_inputs), (song_labels, tag_labels) = inputs, labels

    X_song = np.ndarray((song_inputs.shape[0], 256), dtype=np.float64)
    y_song = song_labels
    for i in range(song_inputs.shape[0]):
        song_input = song_inputs[i]
        song_nonzero = np.count_nonzero(song_input, axis=0)
        
        song_doc = [dataset.idx2song[song_idx] for song_idx in song_input[:song_nonzero]]
        X_song[i,:] = dataset.song_tag_d2v.infer_vector(song_doc)

    return ids, X_song, y_song

def preprocess_tags(ids, inputs, labels):
    (song_inputs, tag_inputs), (song_labels, tag_labels) = inputs, labels

    X_tag = np.ndarray((tag_inputs.shape[0], 256), dtype=np.float64)
    y_tag = tag_labels
    for i in range(tag_inputs.shape[0]):
        tag_input = tag_inputs[i]
        tag_nonzero = np.count_nonzero(tag_input, axis=0)
        
        tag_doc = [dataset.idx2tag[tag_idx] for tag_idx in tag_input[:tag_nonzero]]
        X_tag[i,:] = dataset.tag_song_d2v.infer_vector(tag_doc)

    return ids, X_tag, y_tag

In [4]:
RESULTS_DIR = '/content/drive/My Drive/MelonPlaylistContinuation/results/'
if not os.path.isdir(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
    
# define model, opt
song_model = K.Sequential()
song_model.add(K.layers.Dense(128, input_shape=(256,)))
song_model.add(K.layers.Activation('relu'))
song_model.add(K.layers.Dense(64))
song_model.add(K.layers.Activation('relu'))
song_model.add(K.layers.Dense(dataset.n_songs))
song_opt = K.optimizers.Adam()

song_ckpt_path = os.path.join(CKPT_DIR, 'song', 'checkpoint')
song_ckpt = tf.train.Checkpoint(optimizer=song_opt, model=song_model)
song_ckpt_manager = tf.train.CheckpointManager(song_ckpt, song_ckpt_path, max_to_keep=1)
if song_ckpt_manager.latest_checkpoint:
    song_ckpt.restore(song_ckpt_manager.latest_checkpoint)

# predict
N = len(dataset.val_plylst_list)
steps_per_epoch = N // BS
if N % BS > 0:
    steps_per_epoch += 1
song_generator = iter(dataset.generate_input(mode = 'val', 
                                              batch_size=BS, 
                                              callback=preprocess_songs))

plylst_list = list()
for step in trange(steps_per_epoch):
    ids, X_song, y_song = next(song_generator)
    logits = song_model(X_song)
    logits_masked = tf.where(y_song == 0, logits, -999)
    
    for i in range(ids.shape[0]):
        id_ = int(ids[i]) 
        logit = logits_masked[i,:]
        _, song_idx = tf.math.top_k(logit, k=100)
        song_idx = song_idx.numpy().tolist()
        song_pred = list(map(lambda idx: str(dataset.idx2song[idx]), song_idx))
        plylst = {'id':id_, 
                  'songs':song_pred}
        plylst_list.append(plylst)

results_path = os.path.join(RESULTS_DIR, 'song_results.json')
json.dump(plylst_list, open(results_path, 'w'))

In [5]:
RESULTS_DIR = '/content/drive/My Drive/MelonPlaylistContinuation/results/'
if not os.path.isdir(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
    
# define model, opt
tag_model = K.Sequential()
tag_model.add(K.layers.Dense(128, input_shape=(256,)))
tag_model.add(K.layers.Activation('relu'))
tag_model.add(K.layers.Dense(64))
tag_model.add(K.layers.Activation('relu'))
tag_model.add(K.layers.Dense(dataset.n_tags))
tag_opt = K.optimizers.Adam()

tag_ckpt_path = os.path.join(CKPT_DIR, 'tag', 'checkpoint')
tag_ckpt = tf.train.Checkpoint(optimizer=tag_opt, model=tag_model)
tag_ckpt_manager = tf.train.CheckpointManager(tag_ckpt, tag_ckpt_path, max_to_keep=1)
if tag_ckpt_manager.latest_checkpoint:
    tag_ckpt.restore(tag_ckpt_manager.latest_checkpoint)

# predict
N = len(dataset.val_plylst_list)
steps_per_epoch = N // BS
if N % BS > 0:
    steps_per_epoch += 1
tag_generator = iter(dataset.generate_input(mode = 'val', 
                                              batch_size=BS, 
                                              callback=preprocess_tags))

plylst_list = list()
for step in trange(steps_per_epoch):
    ids, X_tag, y_tag = next(tag_generator)
    logits = tag_model(X_tag)
    logits_masked = tf.where(y_tag == 0, logits, -999)
    
    for i in range(ids.shape[0]):
        id_ = int(ids[i]) 
        logit = logits_masked[i,:]
        _, tag_idx = tf.math.top_k(logit, k=100)
        tag_idx = tag_idx.numpy().tolist()
        tag_pred = list(map(lambda idx: str(dataset.idx2tag[idx]), tag_idx))
        plylst = {'id':id_, 
                  'tags':tag_pred}
        plylst_list.append(plylst)

results_path = os.path.join(RESULTS_DIR, 'tag_results.json')
json.dump(plylst_list, open(results_path, 'w'))

In [8]:
from collections import defaultdict

In [13]:
song_results = json.load(open(os.path.join(RESULTS_DIR, 'song_results.json')))
tag_results = json.load(open(os.path.join(RESULTS_DIR, 'tag_results.json')))

temp = defaultdict(dict)
for song_result, tag_result in zip(song_results, tag_results):
    song_id = song_result['id']
    tag_id = tag_result['id']
    temp[song_id]['songs'] = song_result['songs']
    temp[tag_id]['tags'] = tag_result['tags']

results = list()
for id_, v in temp.items():
    result = {'id': id_,
              'songs': v['songs'], 
              'tags':v['tags']}
    results.append(result)

json.dump(results, open(os.path.join(RESULTS_DIR, 'results.json'), 'w'))